<a href="https://colab.research.google.com/github/AgarwalMayank2/Face_Detection/blob/main/Decision_Tree_PRMLproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task : Identify Face in image using Decision Tree algorithm

Dataset : https://www.kaggle.com/datasets/jessicali9530/lfw-dataset?select=peopleDevTrain.csv

we will grayscale image for scaling down the input size in our model

# assuming each pixel as separate feature

In [ ]:
#write code here

# Using different approch for Decision Tree algo

In [ ]:
#write code here